# Example of `InferenceData` schema in PyStan
The description of the `InferenceData` structure can be found [here](schema.html).

In [1]:
import arviz as az
import pystan
import pandas as pd
import numpy as np
import xarray
xarray.set_options(display_style="html");

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time_since_joined = data.time.values
slack_comments = data.comments.values
names = data.index.values
N = len(names)
data

,comments,time
Alice,7500,4.5
Bob,10100,6.0
Cole,18600,7.0
Danielle,25200,12.0
Erika,27500,18.0


In [3]:
# data for out of sample predictions
candidate_devs = ["Francis", "Gerard"]
candidate_devs_time = np.array([3.6, 5.1])
N_pred = len(candidate_devs)

In [4]:
linreg_prior_code = """
data {
  int<lower=0> N;
  real time_since_joined[N];
}

generated quantities {
    real b0;
    real b1;
    real log_sigma;
    real<lower=0> sigma;
    vector[N] slack_comments_hat;
    
    b0 = normal_rng(0,10);
    b1 = normal_rng(0,10);
    sigma = abs(normal_rng(0,30));
    log_sigma = log(sigma);
    for (i in 1:N) {
        slack_comments_hat[i] = normal_rng(b0 + b1 * time_since_joined[i], sigma);
    }
}
"""
sm_prior = pystan.StanModel(model_code=linreg_prior_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b2d530337b2d60d5322cb10cbc01e530 NOW.


In [5]:
linreg_prior_data_dict = {"N": N, "time_since_joined": time_since_joined}
prior = sm_prior.sampling(data=linreg_prior_data_dict, iter=150, chains=1, algorithm='Fixed_param', warmup=0)

In [6]:
linreg_code = """
data {
  int<lower=0> N;
  real time_since_joined[N];
  real slack_comments[N];
  
  // out of sample prediction
  int<lower=0> N_pred;
  real time_since_joined_pred[N_pred];
}

parameters {
  real b0;
  real b1;
  real log_sigma;
}

transformed parameters {
  real<lower=0> sigma;
  sigma = exp(log_sigma);
}

model {
  b0 ~ normal(0,10);
  b1 ~ normal(0,10);
  sigma ~ normal(0,30);
  for (i in 1:N) {
    slack_comments[i] ~ normal(b0 + b1 * time_since_joined[i], sigma);
  }
  
}

generated quantities {
    vector[N] log_lik;
    vector[N] slack_comments_hat;
    vector[N_pred] slack_comments_pred; // out of sample prediction
    for (i in 1:N) {
        log_lik[i] = normal_lpdf(slack_comments[i] | b0 + b1 * time_since_joined[i], sigma);
        slack_comments_hat[i] = normal_rng(b0 + b1 * time_since_joined[i], sigma);
    }
    // out of sample prediction
    for (i in 1:N_pred) {
        slack_comments_pred[i] = normal_rng(b0 + b1 * time_since_joined_pred[i], sigma);
    }
}
"""
sm = pystan.StanModel(model_code=linreg_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f2b6a9112d927964b65d8878609636d8 NOW.


In [7]:
linreg_data_dict = {"N": N, "slack_comments": slack_comments, "time_since_joined": time_since_joined, "N_pred" : N_pred, "time_since_joined_pred" : candidate_devs_time}
posterior = sm.sampling(data=linreg_data_dict, iter=200, chains=4)

In [8]:
idata_stan = az.from_pystan(
    posterior=posterior,
    prior=prior,
    posterior_predictive="slack_comments_hat",
    prior_predictive="slack_comments_hat",
    observed_data=["slack_comments"],
    constant_data=["time_since_joined"],
    log_likelihood="log_lik",
    predictions=["slack_comments_pred"],
    predictions_constant_data=["time_since_joined_pred"],
    coords={"developer": names, "candidate developer" : candidate_devs},
    dims={
        "slack_comments": ["developer"],
        "log_lik": ["developer"],
        "slack_comments_hat": ["developer"],
        "time_since_joined": ["developer"],
        "slack_comments_pred" : ["candidate developer"],
        "time_since_joined_pred" : ["candidate developer"],
    }
)

In [9]:
idata_stan

Inference data with groups:
	> posterior
	> sample_stats
	> log_likelihood
	> posterior_predictive
	> predictions
	> prior
	> sample_stats_prior
	> prior_predictive
	> observed_data
	> constant_data
	> predictions_constant_data

In this example, each variable has as dimension a combination of the following 3: `chain`, `draw` and `developer`. Moreover, each dimension has specific coordinate values. In the case of `chain` and `draw` it is an integer identifier starting at `0`; in the case of `developer` dimension, its coordinate values are the following strings: `["Alice", "Bob", "Cole", "Danielle", "Erika"]`.

In [10]:
idata_stan.posterior

<xarray.Dataset>
Dimensions:    (chain: 4, draw: 100)
Coordinates:
  * chain      (chain) int32 0 1 2 3
  * draw       (draw) int32 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables:
    b0         (chain, draw) float64 15.63 22.83 26.09 ... 9.892 11.14 8.458
    b1         (chain, draw) float64 76.89 73.55 89.86 ... 80.33 86.35 88.81
    log_sigma  (chain, draw) float64 7.02 7.012 7.022 7.011 ... 7.029 7.007 7.03
    sigma      (chain, draw) float64 1.118e+03 1.11e+03 ... 1.105e+03 1.13e+03
Attributes:
    created_at:                 2020-01-27T20:51:24.018479
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [11]:
idata_stan.sample_stats

<xarray.Dataset>
Dimensions:      (chain: 4, draw: 100)
Coordinates:
  * chain        (chain) int32 0 1 2 3
  * draw         (draw) int32 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables:
    accept_stat  (chain, draw) float64 0.9489 0.9656 0.9993 ... 0.7751 0.9451
    stepsize     (chain, draw) float64 0.01411 0.01411 ... 0.01486 0.01486
    treedepth    (chain, draw) int64 7 10 10 10 10 10 9 10 ... 7 10 10 9 10 9 10
    n_leapfrog   (chain, draw) int64 155 1023 1023 1023 ... 779 1023 791 1023
    diverging    (chain, draw) bool False False False ... False False False
    energy       (chain, draw) float64 1.457e+03 1.458e+03 ... 1.457e+03
    lp           (chain, draw) float64 -1.456e+03 -1.457e+03 ... -1.457e+03
Attributes:
    created_at:                 2020-01-27T20:51:24.027050
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [12]:
idata_stan.posterior_predictive

<xarray.Dataset>
Dimensions:             (chain: 4, developer: 5, draw: 100)
Coordinates:
  * chain               (chain) int32 0 1 2 3
  * draw                (draw) int32 0 1 2 3 4 5 6 7 ... 92 93 94 95 96 97 98 99
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 1.03e+03 ... 1.277e+03
Attributes:
    created_at:                 2020-01-27T20:51:24.039167
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [13]:
idata_stan.predictions

<xarray.Dataset>
Dimensions:              (candidate developer: 2, chain: 4, draw: 100)
Coordinates:
  * chain                (chain) int32 0 1 2 3
  * draw                 (draw) int32 0 1 2 3 4 5 6 7 ... 93 94 95 96 97 98 99
  * candidate developer  (candidate developer) <U7 'Francis' 'Gerard'
Data variables:
    slack_comments_pred  (chain, draw, candidate developer) float64 -713.9 ... 1.78e+03
Attributes:
    created_at:                 2020-01-27T20:51:24.040731
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [14]:
idata_stan.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) int64 7500 10100 18600 25200 27500
Attributes:
    created_at:                 2020-01-27T20:51:24.004465
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [15]:
idata_stan.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2020-01-27T20:51:24.009468
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [16]:
idata_stan.predictions_constant_data

<xarray.Dataset>
Dimensions:                 (candidate developer: 2)
Coordinates:
  * candidate developer     (candidate developer) <U7 'Francis' 'Gerard'
Data variables:
    time_since_joined_pred  (candidate developer) float64 3.6 5.1
Attributes:
    created_at:                 2020-01-27T20:51:24.012470
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [17]:
idata_stan.prior

<xarray.Dataset>
Dimensions:    (chain: 1, draw: 150)
Coordinates:
  * chain      (chain) int32 0
  * draw       (draw) int32 0 1 2 3 4 5 6 7 ... 142 143 144 145 146 147 148 149
Data variables:
    b0         (chain, draw) float64 0.3811 13.03 6.564 ... -8.64 -6.783 -8.146
    b1         (chain, draw) float64 3.634 1.88 -13.2 ... -0.251 -3.334 15.69
    log_sigma  (chain, draw) float64 3.35 3.677 3.218 ... 3.332 1.994 3.957
    sigma      (chain, draw) float64 28.51 39.52 24.98 22.9 ... 27.99 7.343 52.3
Attributes:
    created_at:                 2020-01-27T20:51:24.044306
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [18]:
idata_stan.sample_stats_prior

<xarray.Dataset>
Dimensions:      (chain: 1, draw: 150)
Coordinates:
  * chain        (chain) int32 0
  * draw         (draw) int32 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149
Data variables:
    accept_stat  (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    created_at:                 2020-01-27T20:51:24.046855
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [19]:
idata_stan.prior_predictive

<xarray.Dataset>
Dimensions:             (chain: 1, developer: 5, draw: 150)
Coordinates:
  * chain               (chain) int32 0
  * draw                (draw) int32 0 1 2 3 4 5 6 ... 144 145 146 147 148 149
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 51.94 -4.117 ... 351.4
Attributes:
    created_at:                 2020-01-27T20:51:24.049444
    inference_library:          pystan
    inference_library_version:  2.19.1.1